In [1]:
import torch
from torch import nn
from d2l import torch as d2l
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.optim as optim

In [4]:
## THIS CODE IS TO CREATE THE TENSOR FOR THE TRAINING DATA ##

changi_df = pd.read_csv('ST5188/Data/Final/Imputation/imp_train_set.csv')

# Pivot to create a time series for each (x, y)
changi_pivot = changi_df.pivot(index=['x', 'y'], columns='Date', values='Value')
#print (changi_pivot)

# Reset index to keep (x, y) as columns
changi_pivot = changi_pivot.reset_index()

# Convert time columns back into a NumPy array
ts_changi = changi_pivot.iloc[:, 2:].values  # Ignore first two columns (x, y)
loc_changi = changi_pivot.iloc[:, :2].values  # Store coordinates

# Standardization with Z-score normalisation
changi_mean_LST = ts_changi.mean()
changi_std_LST = ts_changi.std()
ts_changi = (ts_changi - changi_mean_LST) / changi_std_LST 

# Convert to PyTorch tensor wiith extra dimension for LST
X_train_tensor_changi = torch.tensor(ts_changi, dtype=torch.float32).unsqueeze(-1)

X_train_changi = X_train_tensor_changi
y_train_changi = X_train_tensor_changi[:, 1:, :]

#### Without dropout

In [ ]:
## THIS CODE IS TO DEFINE THE LSTM MODEL ##

class LSTMPredictor(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=3, output_size=1):
        super(LSTMPredictor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)  # Fully connected layer

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # LSTM output
        out = self.fc(lstm_out)  # Fully connected layer for final prediction
        return out

In [6]:
## THIS CODE IS TO TRAIN THE MODEL ##
torch.manual_seed(5188)

model = LSTMPredictor() # Model
criterion = nn.MSELoss() # Loss function to track performance over epochs
optimizer = optim.Adam(model.parameters(), lr=0.001) # Adam optimizer (can be switched)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    predictions = model(X_train_changi)

    loss = criterion(predictions[:, :-1, :], y_train_changi)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0: # Print loss every 10 epochs
        print(f"Epoch {epoch}/{num_epochs}, Loss: {loss.item():.4f}")

Epoch 0/100, Loss: 1.0082
Epoch 10/100, Loss: 0.9866
Epoch 20/100, Loss: 0.9414
Epoch 30/100, Loss: 0.9111
Epoch 40/100, Loss: 0.9006
Epoch 50/100, Loss: 0.8959
Epoch 60/100, Loss: 0.8907
Epoch 70/100, Loss: 0.8838
Epoch 80/100, Loss: 0.8766
Epoch 90/100, Loss: 0.8602


In [7]:
## THIS CODE IS FOR FORECASTING##

model.eval()
torch.manual_seed(5188)

with torch.no_grad():
    X_pred_changi = model(X_train_changi)  # Forecast next steps
    X_pred_changi= X_pred_changi[:, -12:, :]  # Extract last 12 bimonthly periods (2023-2024)

# Convert predictions to a NumPy array
X_pred_changi_np = X_pred_changi.squeeze().cpu().numpy()

# **Denormalize predictions**
X_pred_changi_np = (X_pred_changi_np * changi_std_LST) + changi_mean_LST  # Convert back to original from scaled values

# Define bimonthly periods
bimonthly_periods = [
    "Jan-Feb 2023", "Mar-Apr 2023", "May-Jun 2023", "Jul-Aug 2023", "Sep-Oct 2023", "Nov-Dec 2023",
    "Jan-Feb 2024", "Mar-Apr 2024", "May-Jun 2024", "Jul-Aug 2024", "Sep-Oct 2024", "Nov-Dec 2024"
]

# Create a long-format DataFrame
changi_pred_df = pd.DataFrame({
    "x": np.repeat(loc_changi[:, 0], len(bimonthly_periods)),  # Use stored locations
    "y": np.repeat(loc_changi[:, 1], len(bimonthly_periods)),  
    "Date": bimonthly_periods * len(loc_changi),
    "Predicted_LST": X_pred_changi_np.flatten()  # Store denormalized values
})

# changi_pred_df.to_csv('changi_pred_long.csv', index=False) # Save to CSV for RMSE

In [9]:
## THIS CODE IS TO CALCULATE RMSE ##

# Load predicted and true values
true_df = changi_pred_df
pred_df = pd.read_csv("../../../Data/Final/TT Split/changi_test_long.csv")

# Merge predicted and actual values
merged_df = true_df.merge(pred_df, on=["x", "y", "Date"], suffixes=("_true", "_pred"))

# Define the mapping between date and forecast step
date_to_horizon = {
    "Jan-Feb 2023": 1,
    "May-Jun 2023": 3,
    "Nov-Dec 2023": 6,
    "May-Jun 2024": 9,
    "Nov-Dec 2024": 12
}

# Filter only relevant dates
selected_dates = list(date_to_horizon.keys())
filtered_df = merged_df[merged_df["Date"].isin(selected_dates)].copy()

# Map forecast horizon
filtered_df["Forecast horizon"] = filtered_df["Date"].map(date_to_horizon)

# Compute RMSE per forecast horizon
rmse_by_horizon = filtered_df.groupby("Forecast horizon").apply(
    lambda g: np.sqrt(np.mean((g["Predicted_LST"] - g["Value"]) ** 2))
).reset_index(name="LSTM w/o dropout")

# Compute overall RMSE
overall_rmse = np.sqrt(np.mean((filtered_df["Predicted_LST"] - filtered_df["Value"]) ** 2))
overall_row = pd.DataFrame({
    "Forecast horizon": ["Overall"],
    "LSTM w/o dropout": [overall_rmse]
})

# Combine RMSEs
final_rmse_df = pd.concat([rmse_by_horizon, overall_row], ignore_index=True)

# Save the table
final_rmse_df.to_csv("rmse_lstm_no_dropout.csv", index=False)

# Show the result
print(final_rmse_df)


  Forecast horizon  LSTM w/o dropout
0                1          5.503483
1                3          1.722315
2                6          2.933474
3                9          1.149421
4               12          3.917198
5          Overall          3.421274


C:\Users\naomi\AppData\Local\Temp\ipykernel_13664\1001673750.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  rmse_by_horizon = filtered_df.groupby("Forecast horizon").apply(


#### With dropout

In [16]:
class LSTMPredictor(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=3, output_size=1, dropout=0.2):
        super(LSTMPredictor, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0.0,  # dropout only works if num_layers > 1
            batch_first=True
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.dropout(lstm_out)
        out = self.fc(out)
        return out

In [ ]:
# Initialize model
torch.manual_seed(5188)
model = LSTMPredictor()

# Define loss function and optimizer with weight decay
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-6)  # Added weight decay

# Training loop
num_epochs = 150
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    predictions = model(X_train_changi)
    predictions = predictions[:, :-1, :]

    loss = criterion(predictions, y_train_changi)
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{num_epochs}, Loss: {loss.item():.4f}")

Epoch 0/200, Loss: 1.0084
Epoch 10/200, Loss: 0.9869
Epoch 20/200, Loss: 0.9423
Epoch 30/200, Loss: 0.9128
Epoch 40/200, Loss: 0.9030
Epoch 50/200, Loss: 0.8990
Epoch 60/200, Loss: 0.8946
Epoch 70/200, Loss: 0.8892
Epoch 80/200, Loss: 0.8835
Epoch 90/200, Loss: 0.8752
Epoch 100/200, Loss: 0.8544
Epoch 110/200, Loss: 0.8375
Epoch 120/200, Loss: 0.8203
Epoch 130/200, Loss: 0.8025
Epoch 140/200, Loss: 0.7769
Epoch 150/200, Loss: 0.7555
Epoch 160/200, Loss: 0.7349
Epoch 170/200, Loss: 0.7215
Epoch 180/200, Loss: 0.7051
Epoch 190/200, Loss: 0.6828


In [27]:
# Forecasting
model.eval()
torch.manual_seed(5188)
with torch.no_grad():
    X_pred_changi = model(X_train_changi)
    X_pred_changi = X_pred_changi[:, -12:, :]

# Denormalize predictions
X_pred_changi_np = (X_pred_changi.squeeze().cpu().numpy() * changi_std_LST) + changi_mean_LST

# Define bimonthly periods
bimonthly_periods = [
    "Jan-Feb 2023", "Mar-Apr 2023", "May-Jun 2023", "Jul-Aug 2023", "Sep-Oct 2023", "Nov-Dec 2023",
    "Jan-Feb 2024", "Mar-Apr 2024", "May-Jun 2024", "Jul-Aug 2024", "Sep-Oct 2024", "Nov-Dec 2024"
]

# Create DataFrame for predictions
changi_pred_df = pd.DataFrame({
    "x": np.repeat(loc_changi[:, 0], len(bimonthly_periods)),
    "y": np.repeat(loc_changi[:, 1], len(bimonthly_periods)),  
    "Date": bimonthly_periods * len(loc_changi),
    "Predicted_LST": X_pred_changi_np.flatten()
})

In [31]:
## THIS CODE IS TO CALCULATE RMSE ##

# Load predicted and true values
true_df = changi_pred_df
pred_df = pd.read_csv("../../../Data/Final/TT Split/changi_test_long.csv")

# Merge predicted and actual values
merged_df = true_df.merge(pred_df, on=["x", "y", "Date"], suffixes=("_true", "_pred"))

# Define the mapping between date and forecast step
date_to_horizon = {
    "Jan-Feb 2023": 1,
    "May-Jun 2023": 3,
    "Nov-Dec 2023": 6,
    "May-Jun 2024": 9,
    "Nov-Dec 2024": 12
}

# Filter only relevant dates
selected_dates = list(date_to_horizon.keys())
filtered_df = merged_df[merged_df["Date"].isin(selected_dates)].copy()

# Map forecast horizon
filtered_df["Forecast horizon"] = filtered_df["Date"].map(date_to_horizon)

# Compute RMSE per forecast horizon
rmse_by_horizon = filtered_df.groupby("Forecast horizon").apply(
    lambda g: np.sqrt(np.mean((g["Predicted_LST"] - g["Value"]) ** 2))
).reset_index(name="LSTM w/o dropout")

# Compute overall RMSE
overall_rmse = np.sqrt(np.mean((filtered_df["Predicted_LST"] - filtered_df["Value"]) ** 2))
overall_row = pd.DataFrame({
    "Forecast horizon": ["Overall"],
    "LSTM w/o dropout": [overall_rmse]
})

# Combine RMSEs
final_rmse_df = pd.concat([rmse_by_horizon, overall_row], ignore_index=True)

# Save the table
final_rmse_df.to_csv("rmse_lstm_dropout.csv", index=False)

# Show the result
print(final_rmse_df)


  Forecast horizon  LSTM w/o dropout
0                1          5.650649
1                3          1.419931
2                6          2.342539
3                9          1.214700
4               12          5.069484
5          Overall          3.649896


C:\Users\naomi\AppData\Local\Temp\ipykernel_13664\98453864.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  rmse_by_horizon = filtered_df.groupby("Forecast horizon").apply(
